### Load the Data

upload a data set with file structure 

data/solo/positive and data/solo/negative

then add empty data/spec/positive and
data/spec/negative directories

In [ ]:
# # Used to unzip a data set loaded to the drive
# import zipfile

# with zipfile.ZipFile("/content/drive/MyDrive/MotifGAN Capstone/combo_spec_1.zip") as z:
#   z.extractall()

### Data Augmentation

In [ ]:
import os
os.system("python3 augment.py")

### convert audio to spectograms


In [ ]:
os.system("python3 create_spectrograms.py")

In [ ]:
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

img_name = random.choice(os.listdir('./data/spec/positive'))
print(img_name)
img = mpimg.imread(f'./data/spec/positive/{img_name}')
plt.imshow(img)
plt.show()

### create the dataset

In [ ]:
import tensorflow as tf

In [ ]:
# Load dataset from directory with keras

# sometimes a directory called .ipynb_checkpoints is present, remove it if so
if os.path.isdir('./data/spec/.ipynb_checkpoints'):
  os.removedirs("./data/spec/.ipynb_checkpoints")

train_directory = './data/spec/'
test_directory = './data/test/'
batch_size = 16

# Validation data needs to be specified at training now. 
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_directory, labels='inferred', label_mode='int', batch_size=batch_size, 
    image_size=(360,225), seed=123, validation_split=0.2, subset="validation")

# test_ds = tf.keras.preprocessing.image_dataset_from_directory(
#     test_directory, labels='inferred', label_mode='int', batch_size=batch_size,
#     image_size=(360,225), seed=123)


class_names = train_ds.class_names
print(class_names)

In [ ]:
# Straight from the microsoft tutorial 
# shows the spectrograms with their labels
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(2):
    for i in range(9):
        ax = plt.subplot(3, 5, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(class_names[labels[i]])
        plt.axis("off")

## ResNet-34 Model

In [ ]:
from tensorflow import keras

In [ ]:
class ResidualUnit(keras.layers.Layer):
    def __init__(self, filters, strides=1, activation="relu", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
        self.main_layers = [
            keras.layers.Conv2D(filters, 3, strides=strides,
                            padding="same", use_bias=False),
            keras.layers.BatchNormalization(),
            self.activation,
            keras.layers.Conv2D(filters, 3, strides=1,
                            padding="same", use_bias=False),
            keras.layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                keras.layers.Conv2D(filters, 1, strides=strides,
                                  padding="same", use_bias=False),
                keras.layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(64, 7, strides=2, input_shape=[360, 225, 3],
                              padding="same", use_bias=False))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Activation("relu"))
model.add(keras.layers.MaxPool2D(pool_size=3, strides=2, padding="same"))
prev_filter = 64
for filters in [64]*3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filter else 2
    model.add(ResidualUnit(filters, strides=strides))
    prev_filters = filters
#model.add(keras.layers.GlobalAvgPool2D())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(2, activation="softmax"))

In [ ]:
learning_rate = 0.02

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
optimizer = tf.keras.optimizers.SGD(learning_rate)
metrics = ['acc']
model.compile(optimizer, loss_fn, metrics)
#model.summary()

In [ ]:
# Set the epochs
epochs = 50
print('\nFitting:')

# Train the model.
history = model.fit(train_ds, epochs=epochs, batch_size=1)

In [ ]:
plt.plot(history.history['accuracy'])
# plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()